In [26]:
# Importing the Adult Dataset
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold

In [27]:
df = pd.read_csv("adult.data", header=None)
headers = {0:"Age",1:"Workclass",2:"fnlwgt", 3:"Education", 4:"Education-num", 5:"Maritial-Status"
              , 6:"Occupation", 7:"Relationship", 8:"Race", 9:"Sex", 10:"Capital-Gain", 11:"Capital-Loss"
              , 12:"Hours-per-Week", 13:"Native-Country", 14:"Class"}
df = df.rename(columns=headers)

# Shape of Dataframe is (32561, 15)
df.head()

,Age,Workclass,fnlwgt,Education,Education-num,Maritial-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-per-Week,Native-Country,Class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [28]:
# One-Hot Encoding of Categorical Variables

# Drops Unecessary Columns
df = pd.get_dummies(df)
df = df.drop(['Workclass_ ?'], axis=1)
df = df.drop(['Class_ >50K'], axis=1)

df.head()

,Age,fnlwgt,Education-num,Capital-Gain,Capital-Loss,Hours-per-Week,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,Workclass_ Private,...,Native-Country_ Puerto-Rico,Native-Country_ Scotland,Native-Country_ South,Native-Country_ Taiwan,Native-Country_ Thailand,Native-Country_ Trinadad&Tobago,Native-Country_ United-States,Native-Country_ Vietnam,Native-Country_ Yugoslavia,Class_ <=50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
3,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
4,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [29]:
# 3) Implement the k-NN.
class simple_KNeighborsClassifier(object):
    def __init__(self, k):
        """
        k-NN initialization.
            k: Number of nearest neighbors.
        """
        self.k = k
        
    def fit(self, X_train, Y_train):
        """
        k-NN fitting function.
            X_train: Feature vectors in training set.
            Y_train: Labels in training set.
        """
        self.X_train = X_train
        self.Y_train = Y_train
        
    def predict(self, X_pred):
        """
        k-NN prediction function.
            X_pred: Feature vectors in training set.
        Return the predicted labels for X_pred. Shape: (len(X_pred), )
        """
        Y_pred = []
        for i in range(len(X_pred)):
            distances = ((self.X_train - X_pred[i].reshape(1,-1)) \
                          ** 2.0).sum(axis = 1)
            distances_and_labels = [(distances[i], self.Y_train[i]) \
                                    for i in range(len(self.X_train))]
            distances_and_labels.sort()
            top_k_labels = np.array(distances_and_labels)[:self.k,1].ravel()
            mode, _ = scipy.stats.mode(top_k_labels)
            Y_pred.append(mode[0])
        return np.array(Y_pred)

In [30]:
# 4) Implement the cross-validation.

def simple_cross_validation(X_train_val, Y_train_val, k, fold):
    """
    A simple cross-validation function for k-NN.
    
    X_train_val: Features for train and val set. 
                 Shape: (num of data points, num of features)
    Y_train_val: Labels for train and val set.
                 Shape: (num of data points,)
    k:           Parameter k for k-NN.
    fold:        The number of folds to do the cross-validation.
    
    Return the average accuracy on validation set.
    """
    val_acc_list = []
    train_acc_list = []
    for i in range(fold):
        start       = int(i*len(X_train_val)/fold)
        end         = int((i+1)*len(X_train_val)/fold)
        index_train = list(range(0, start)) + list(range(end, len(X_train_val)))
        index_val   = range(start, end)
        X_train     = X_train_val[index_train]
        Y_train     = Y_train_val[index_train]
        X_val       = X_train_val[index_val]
        Y_val       = Y_train_val[index_val]
        #classifier  = KNeighborsClassifier(algorithm='brute', n_neighbors=k)
        classifier = simple_KNeighborsClassifier(k=k)
        classifier.fit(X_train, Y_train)
        val_acc_list.append(sum(classifier.predict(X_val) == Y_val) / len(X_val))
        train_acc_list.append(sum(classifier.predict(X_train) == Y_train) / len(X_train))
        
    return sum(val_acc_list) / len(val_acc_list), \
           sum(train_acc_list) / len(train_acc_list)

In [31]:
# 5) Implement the grid search function.

def simple_GridSearchCV_fit(X_train_val, Y_train_val, k_list, fold):
    """
    A simple grid search function for k with cross-validation in k-NN.
    
    X_train_val: Features for train and val set. 
                 Shape: (num of data points, num of features)
    Y_train_val: Labels for train and val set.
                 Shape: (num of data points,)
    k_list:      The list of k values to try.
    fold:        The number of folds to do the cross-validation.
    
    Return the val and train accuracy matrix of cross-validation. 
    All combinations of k are included in the array. 
    Shape: (len(k_list), )
    """
    val_acc_array = np.zeros(len(k_list))
    train_acc_array = np.zeros(len(k_list))
    for i in range(len(k_list)):
        val_acc_array[i], train_acc_array[i] = simple_cross_validation(
            X_train_val, Y_train_val, k_list[i], fold)
    return val_acc_array, train_acc_array

In [32]:
def draw_heatmap_knn(acc, acc_desc, k_list):
    plt.figure(figsize = (2,4))
    ax = sns.heatmap(acc, annot=True, fmt='.3f', yticklabels=k_list, xticklabels=[])
    ax.collections[0].colorbar.set_label("accuracy")
    ax.set(ylabel='$k$')
    plt.title(acc_desc + ' w.r.t $k$')
    sns.set_style("whitegrid", {'axes.grid' : False})
    plt.show()

In [ ]:
# Setting up to split 
df_array = df.as_matrix()
X = df_array[:, 0:-1] #Shape: (32561,107)
y = df_array[:, -1]   #Shape: (32561,)
#k_list = [1,2,3,4,5,6,7,8,9,10
 #         ,11,12,13,14,15,16,17,18,19,20
 #         ,21,22,23,24,25,26]
k_list = [1,2,4,8,12,16,20,24,26]
split_per = [.2, .5, .8]
split_train = []
split_test = []
best_params = []

In [ ]:
# 6) Perform grid search.
NUM_TRIALS = 3

for per in split_per:
    val_acc = []
    train_acc = []
    for i in range(0,NUM_TRIALS):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.8)
        val_acc_array, train_acc_array = \
            simple_GridSearchCV_fit(X_train, y_train, k_list, 5)
        val_acc.append(val_acc_array)
        train_acc.append(train_acc_array)
    


In [59]:
score_avg = []
scores_averaged = np.empty([k_list_length])
k_list_length = 4
trials = 3
one = [1, 2, 3, 4]
two = [5, 6, 7, 8]
three = [9, 10, 11, 12]

score_avg.append(one)
score_avg.append(two)
score_avg.append(three)

for i in range(0,k_list_length):
    avg_array = np.empty([trials])
    for j in range(0,trials):
        avg_array[j] = score_avg[j][i]
    print (avg_array)
    scores_averaged[i] = np.mean(avg_array)

[ 1.  5.  9.]
[  2.   6.  10.]
[  3.   7.  11.]
[  4.   8.  12.]
[ 5.  6.  7.  8.]
